In [121]:
%run WebMiningPython.py

Extract information from Rational webpage

Company: 2414
Type: 2
Date: 2020-01-08
Title: Roadshow
City: 1
Institute: Not Available


Company: 2414
Type: 1
Date: 2020-01-09
Title: Conference
City: 246
Institute: 40288251267396ba01267a4b6d8100fb


Company: 2414
Type: 1
Date: 2020-01-21
Title: Conference
City: 2
Institute: Not Available


Company: 2414
Type: 1
Date: 2020-03-19
Title: Conference
City: 3
Institute: Not Available


Company: 2414
Type: 1
Date: 2020-03-26
Title: Conference
City: Not Available
Institute: 8a828487499354c3014d6770452357f9


Company: 2414
Type: 1
Date: 2020-03-31
Title: Conference
City: 38
Institute: Not Available


Company: 2414
Type: 2
Date: 2020-04-22
Title: Roadshow
City: 2
Institute: Not Available


Company: 2414
Type: 1
Date: 2020-05-13
Title: Conference
City: 187
Institute: 8a828487499354c3014a4e5fc6a842d7


Company: 2414
Type: 1
Date: 2020-05-14
Title: Conference
City: 146
Institute: 8a828487499354c3014a4e5fc6a842d7


Company: 2414
Type: 1
Date: 2020-06-03

In [2]:
import re
from dateutil.parser import parse
from bs4 import BeautifulSoup
from urllib import request
import time

def altnames(mydict):
    aux_mydict = mydict.copy()
    for elem in mydict.keys():
        if "/" in elem:
            for alt in elem.split("/"):
                aux_mydict[alt] = mydict[elem]
            del aux_mydict[elem]
    return aux_mydict  


############# To be modified #############
##currentdate=time.strftime('%Y-%m-%d')
currentdate='2019-01-01'
TOPIC = "[Cc]onference|[Rr]oadshow|[Ii]nvestor days?"
pagename="Rational"
ID="2414"
address="https://www.rational-online.com/en_in/Company/Investor_Relations/Financial_and_roadshow_calendar_"
##########################################

#the_dict = {"[Rr]oadshow":1,""}

In [3]:
cities=dict([line.strip().split(";") for line in open("city.csv", encoding='ISO-8859-1').readlines()[1:]])

In [4]:
cities["1"]

'Amsterdam'

In [5]:
cities["4"]

'Zurich/Zürich'

In [6]:
pcities = dict(zip([x for x in cities.values()],cities.keys()))
pcities=altnames(pcities)

In [7]:
pcities["Amsterdam"]

'1'

In [8]:
pcities["Zurich"],pcities["Zürich"]

('4', '4')

In [9]:
sponsors=dict([line.strip().split(";") for line in open("sponsor.csv").readlines()[1:]])
psponsors = dict(zip(sponsors.values(),sponsors.keys()))
psponsors = altnames(psponsors)

In [10]:
#!pip install lxml

In [48]:
url=request.urlopen(address,timeout=None).read()
soup = BeautifulSoup(url,"lxml")

In [40]:
len(soup.find_all("table"))

4

In [50]:
tab = soup.find_all("table")[2]

In [42]:
tab


<table class="table-standard table-mobile" width="100%">
<tbody>
<tr>
<td class="m-head">Event</td>
<td>Preliminary Figures Fiscal Year 2019</td>
</tr>
<tr>
<td class="m-head">Location</td>
<td>Landsberg am Lech</td>
</tr>
<tr>
<td class="m-head">Date</td>
<td>06 Feb 2020</td>
</tr>
<tr><td class="t-border"></td><td class="t-border"></td></tr>
<tr>
<td class="m-head">Event</td>
<td>Financial Figures Fiscal Year 2019</td>
</tr>
<tr>
<td class="m-head">Location</td>
<td>Hamburg</td>
</tr>
<tr>
<td class="m-head">Date</td>
<td>17 Mar 2020</td>
</tr>
<tr><td class="t-border"></td><td class="t-border"></td></tr>
<tr>
<td class="m-head">Event</td>
<td>Telephone Conference Fiscal Year 2019</td>
</tr>
<tr>
<td class="m-head">Location</td>
<td>Hamburg</td>
</tr>
<tr>
<td class="m-head">Date</td>
<td>17 Mar 2020</td>
</tr>
<tr><td class="t-border"></td><td class="t-border"></td></tr>
<tr>
<td class="m-head">Event</td>
<td>Balance Sheet Press Conference Fiscal Year 2019</td>
</tr>
<tr>
<td class=

In [43]:
tab.find_all("span",{"class":"givacolor"})

[]

In [44]:
tab.find_all("td")

[<td class="m-head">Event</td>,
 <td>Preliminary Figures Fiscal Year 2019</td>,
 <td class="m-head">Location</td>,
 <td>Landsberg am Lech</td>,
 <td class="m-head">Date</td>,
 <td>06 Feb 2020</td>,
 <td class="t-border"></td>,
 <td class="t-border"></td>,
 <td class="m-head">Event</td>,
 <td>Financial Figures Fiscal Year 2019</td>,
 <td class="m-head">Location</td>,
 <td>Hamburg</td>,
 <td class="m-head">Date</td>,
 <td>17 Mar 2020</td>,
 <td class="t-border"></td>,
 <td class="t-border"></td>,
 <td class="m-head">Event</td>,
 <td>Telephone Conference Fiscal Year 2019</td>,
 <td class="m-head">Location</td>,
 <td>Hamburg</td>,
 <td class="m-head">Date</td>,
 <td>17 Mar 2020</td>,
 <td class="t-border"></td>,
 <td class="t-border"></td>,
 <td class="m-head">Event</td>,
 <td>Balance Sheet Press Conference Fiscal Year 2019</td>,
 <td class="m-head">Location</td>,
 <td>Hamburg</td>,
 <td class="m-head">Date</td>,
 <td>17 Mar 2020</td>,
 <td class="t-border"></td>,
 <td class="t-border"></t

In [45]:
tab.find_all("td")[1]

<td>Preliminary Figures Fiscal Year 2019</td>

In [46]:
tab.find_all("td")[1].text

'Preliminary Figures Fiscal Year 2019'

In [47]:
for row in tab:
    for e in row.recursiveChildGenerator():
        if isinstance(e,str):
            print("++++++++++++++++++")
            print(e)

AttributeError: 'NavigableString' object has no attribute 'recursiveChildGenerator'

In [106]:
the_table = soup.find_all("table")[2].find_all("tr")

for row in the_table:
    
    line = [elem.text.strip() for elem in row.find_all("td")]
    line = "$".join(line)
    
    m = re.search(TOPIC,line)
    
    if m != None:
        
        r = "(.*)\$(.*)\$(.*)\$(.*)"
        n = re.search(r,line)
        
        if n.group(1).lower() == "conference":
            t = 1
        elif n.group(1).lower() == "roadshow":
            t = 2
        else:
            t = 3
        
        date = n.group(4)
        date = parse(date,dayfirst = True).strftime('%Y-%m-%d')

        if n.group(2) in pcities.keys():
            c = pcities[n.group(2)]
        
        if n.group(3) in psponsors.keys():
            s = psponsors[n.group(3)]
        else:
            s = "Not Available"
        
        print("Company:",ID)
        print("Type:",t)
        print("Date:",date)
        print("Title:",n.group(1))
        print("City:",c)
        print("Institute:",s)
        
        print("")

Company: 2414
Type: 2
Date: 2020-01-08
Title: Roadshow
City: 1
Institute: Not Available

Company: 2414
Type: 1
Date: 2020-01-09
Title: Conference
City: 246
Institute: 40288251267396ba01267a4b6d8100fb

Company: 2414
Type: 1
Date: 2020-01-21
Title: Conference
City: 2
Institute: Not Available

Company: 2414
Type: 1
Date: 2020-03-19
Title: Conference
City: 3
Institute: Not Available

Company: 2414
Type: 1
Date: 2020-03-26
Title: Conference
City: 3
Institute: 8a828487499354c3014d6770452357f9

Company: 2414
Type: 1
Date: 2020-03-31
Title: Conference
City: 38
Institute: Not Available

Company: 2414
Type: 2
Date: 2020-04-22
Title: Roadshow
City: 2
Institute: Not Available

Company: 2414
Type: 1
Date: 2020-05-13
Title: Conference
City: 187
Institute: 8a828487499354c3014a4e5fc6a842d7

Company: 2414
Type: 1
Date: 2020-05-14
Title: Conference
City: 146
Institute: 8a828487499354c3014a4e5fc6a842d7

Company: 2414
Type: 1
Date: 2020-06-03
Title: Conference
City: 22
Institute: ff808081227ad9f7012298b2b

In [68]:
row  = soup.find_all("table")[2].find_all("tr")[1]
row

<tr>
<td colspan="1" rowspan="1"><span class="my-ui-table-cell-label">Roadshow</span></td>
<td colspan="1" rowspan="1"><span class="my-ui-table-cell-label">Amsterdam</span></td>
<td colspan="1" rowspan="1"><span class="my-ui-table-cell-label">KeplerCheuvreux</span></td>
<td colspan="1" rowspan="1"><span class="my-ui-table-cell-label">08 Jan 2020</span></td>
</tr>

In [70]:
line=[elem.text.strip() for elem in row.find_all("td")]
line

['Roadshow', 'Amsterdam', 'KeplerCheuvreux', '08 Jan 2020']

In [71]:
line = "$".join(line)
line

'Roadshow$Amsterdam$KeplerCheuvreux$08 Jan 2020'

In [72]:
m1 = re.search(TOPIC,line)
print(m1 != None)

True


In [73]:
m1.group(0)

'Roadshow'

In [108]:
r="(.*)\$(.*)\$(.*)\$(.*)"
m2=re.search(r,line)
m2

<re.Match object; span=(0, 35), match='Conference$Hamburg$Warburg$Jun 2020'>

In [109]:
m2.group(0),m2.group(1),m2.group(2),m2.group(3),m2.group(4)

('Conference$Hamburg$Warburg$Jun 2020',
 'Conference',
 'Hamburg',
 'Warburg',
 'Jun 2020')

In [76]:
date=m2.group(3)
date

'08 Jan 2020'

In [77]:
parse(date,dayfirst=True).strftime('%Y-%m-%d')

'2020-01-08'

In [30]:
parse('10 January 2019',dayfirst=True).strftime('%Y-%m-%d')

'2019-01-10'

In [111]:
r="\d{1,2}(/\d{1,2})? [A-Z][a-z]+ \d{4}"

In [112]:
dm=re.search(r,date)
dm

In [113]:
firstPart=dm.group(0)
firstPart = re.sub("/\d{1,2}","",firstPart)
firstPart

AttributeError: 'NoneType' object has no attribute 'group'

In [114]:
dm.group(1)

AttributeError: 'NoneType' object has no attribute 'group'

In [115]:
if dm.group(1):
    print(parse(firstPart).strftime('%Y-%m-%d')+dm.group(1)) 

AttributeError: 'NoneType' object has no attribute 'group'

In [116]:
x = 1.10 * 10**2
y = .085 * 10**2
x - y 

101.50000000000001